# Day 8: Treetop Tree House

How many trees are visible from outside the grid?

* A tree is visible if all the trees between it and the edge are shorter.
* Each number \[0-9\] in the input represents the height of a tree.

## Approach

Trees are seen by looking in from the outside. So start by looking into the forest from each position along the perimeter. For the _F_ x _F_ forest:

```
     0    1     2    ...  F-1   F
0   🌲   🌳   🌳        🌲   🌳
1   🌲   🌲   🌳        🌳   🌲
2   🌲   🌳   🎄        🌳   🌳
...
F-1 🌲   🌳   🌳        🌲   🌳 
F   🌳   🌳   🌳        🌲   🌲
```

The trees in column 0, column _F_, row 0 and row _F_ are all visible. So we initiailize with that tree as the first visible when looking into the row or column. There's a further optimization that's pretty clear though. If we start with the top edge looking down the second column then find that the first _D_ trees are all visible then we don't need to consider them when we later look in along rows 0 - _D_. There's a tricky detail here - we don't need to re-evaluate a tree that's known to be **visible** but the only way we know a tree is **not visible** is that it's no taller than any tree which shares a row or column.

One way to remember which trees are visible would be to make an _F_ x _F_ grid of bools called _V_ to track visibility and consult that when starting each search down a row or column. If the tree height comparison were expensive then this also gives a bool check which would be less expensive than repeating the tree height comparison.

Another option is to follow a spiral around the perimeter of the forest moving inward. We're not just looking for a hull around the outside though - it's possible to look inward and see the first 2 trees, not see the next 3, but see the 6th tree if they have heights like 2, 4, 1, 1, 1, 5. The spiral search is appealing since the inner search boundary will still probably shrink over time and the edges can be skipped. It's probably easy to mess this optimization up. However with a simple look in from each edge we know we can stop if we hit a tree of height 9. That's a cheap optimization that might be worth it. I'll stick with the simpler method of looking in from each edge.

Make 4 lists: _N_, _E_, _S_, _W_. Each list stores the hieght of the tallest tree visible when looking into the forest from that edge. For row 0, _N_[0] = the top row of the forest. Do the equivalent for _E_, _S_, and _W_. Then start on the north side for column 1:

1. Iterate over trees in row _r_.
2. Comparing the height to _N_\[_c_\]. If it's taller, update _N_ and _V_.
3. Repeat for each column.

Do the equivalent looking up from the south side, inverting the search direction. Add a guard to check if the current tree is already marked visible in _V_.

In [1]:
from dataclasses import dataclass
from typing import List
import os

In [2]:
class Forest():

    def __init__(self, trees: List[List[bool]]):
        self.trees_ : List[List[int]] = trees.copy()
        self.visibility_ : Optional(List[List[bool]]) = None
    
    def trees(self) -> List[List[int]]:
        return self.trees_

    def edge_len(self) -> int:
        return len(self.trees_[0])

    def visibility(self) -> List[List[bool]]:
        if self.visibility_:
            return self.visibility_
        #self.visibility_ = self.edge_len() * [[True, *((self.edge_len() - 2) * [False]), True]]
        #self.visibility_[0] = self.edge_len() * [True]
        #self.visibility_[self.edge_len() - 1] = self.edge_len() * [True]
        edge_len = self.edge_len()
        v = []
        for r in range(0, edge_len):
            row = [True]
            for c in range(1, edge_len - 1):
                row.append(False)
            row.append(True)
            v.append(row)
        for c in range(0, edge_len):
            v[0][c] = True
            v[edge_len - 1][c] = True
        self.visibility_ = v
        # Track the tallest tree seen looking from each direction.
        north : List[int] = self.row(0)
        east : List[int] = self.column(self.edge_len()-1)
        south : List[int] = self.row(self.edge_len()-1)
        west : List[int] = self.column(0)
        # Skip the first and last rows because they are done.
        for r in range(1, edge_len - 1):
            # Skip the first and last columns because they are done.
            for c in range(1, edge_len - 1):
                # TODO: Seems like we could check visibility here and bail out 
                # early if it's set. Only one tree can be visible from both directions
                tree = self.trees_[r][c]
                if tree > north[c]:
                    north[c] = tree
                    self.visibility_[r][c] = True
        # From the south looking north.
        for r in range(edge_len - 2, 0, -1):
            for c in range(1, edge_len - 1):
                tree = self.trees_[r][c]
                if tree > south[c]:
                    south[c] = tree
                    self.visibility_[r][c] = True
        # From the east looking west.
        for c in range(edge_len - 2, 0, -1):
            for r in range(1, edge_len - 1):
                tree = self.trees_[r][c]
                if tree > east[r]:
                    east[r] = tree
                    self.visibility_[r][c] = True
        # From the west looking east.
        for c in range(1, edge_len - 1):
            for r in range(1, edge_len - 1):
                tree = self.trees_[r][c]
                if tree > west[r]:
                    west[r] = tree
                    self.visibility_[r][c] = True
        return self.visibility_

    def row(self, index : int) -> List[int]:
        # Return a copy to be consistent with `column()`
        return self.trees_[index].copy()

    def column(self, index : int) -> List[int]:
        return self.trees_[index] 

    

In [3]:
def get_testdata():
    return [os.path.join('testdata',f) for f in ['easy1.txt', 'easy2.txt', 'easy3.txt', 'easy4.txt', 'sample1.txt']]

In [4]:
def load_data(filename : str):
    rows : List[List[int]] = []
    with open(filename) as f:
        for line in f.readlines():
            rows.append([int(i) for i in line.strip()])
    return Forest(rows)

In [5]:
def exercise():
    for f in get_testdata():
        forest = load_data(f)
        for l in forest.trees():
            print(f'{l}')
        for l in forest.visibility():
            print(f'{l}')

exercise()

[1, 1, 1, 1]
[1, 0, 0, 1]
[1, 0, 0, 1]
[1, 1, 1, 1]
[True, True, True, True]
[True, False, False, True]
[True, False, False, True]
[True, True, True, True]
[4, 4, 4, 4]
[3, 3, 3, 3]
[2, 2, 2, 2]
[1, 1, 1, 1]
[True, True, True, True]
[True, True, True, True]
[True, True, True, True]
[True, True, True, True]
[1, 1, 1, 1, 1]
[1, 9, 9, 9, 1]
[1, 9, 0, 9, 1]
[1, 9, 9, 9, 1]
[1, 1, 1, 1, 1]
[True, True, True, True, True]
[True, True, True, True, True]
[True, True, False, True, True]
[True, True, True, True, True]
[True, True, True, True, True]
[1, 4, 1, 4, 1]
[4, 2, 4, 2, 4]
[3, 3, 1, 3, 3]
[4, 4, 4, 4, 4]
[5, 5, 5, 5, 5]
[True, True, True, True, True]
[True, False, True, False, True]
[True, True, False, False, True]
[True, False, False, False, True]
[True, True, True, True, True]
[6, 5, 6, 6, 2, 5, 5, 5, 5]
[3, 2, 6, 6, 4, 3, 3, 6, 6]
[3, 3, 3, 3, 3, 5, 3, 4, 6]
[2, 2, 2, 6, 4, 2, 3, 3, 6]
[5, 4, 2, 3, 4, 6, 3, 5, 2]
[4, 4, 5, 6, 2, 4, 4, 6, 3]
[2, 2, 4, 6, 5, 5, 5, 2, 2]
[2, 6, 6, 2, 2, 3,

# Solve part 1

In [6]:
def solve():
    forest = load_data('input.txt')
    v = forest.visibility()
    return sum([sum(r) for r in v])

print(solve())

1803
